From Wageningen B-series open water propeller performance charts (https://doi.org/10.5281/zenodo.8352831):

As early as 1883, R.E. Froude, after extensive experiments with model propellers behind ship models, and model propellers separate from the ship, found that a propeller behind a ship, rotating at a rotational velocityn , behaves like a propeller separate from the ship, operating at the same rotational velocityn , at a reduced advance velocity $V_0 < V$ , called ‘effective advance velocity’. He then introduced the ‘effective wake fraction’w and the ‘relative rotative efficiency $\eta_R$ as the factors relating the two conditions, according to the following equations (thrust equalization method is assumed, Taylor’s definition of wake fraction $W$ is used in (7)):

$$
\begin{aligned}
T_0 &= T (5) \\
Q_0 &= Q \eta_R (6) \\
V_0 &= V (1 - W) (7)
\end{aligned}
$$

where $T_0$, $V_0$, $Q_0$ denote the propeller thrust, effective speed of advance and torque of the propeller operating separate from the ship and, $T$, $V$, $Q$ the propeller thrust,
ship’s speed and torque of the propeller operating behind ship.

We use the following notation for the dimensionless variables for the propeller operation separate from the ship (open water or free running conditions):

$$
\begin{aligned}
J_0 &= V_0 / (n D) : advance coefficient, open-water (8) \\
k_{t0} &= T_0 / (\rho n^2 D^4) : thrust coefficient, open-water (9) \\
k_{q0} &= Q_0 / (\rho n^2 D^5): torque coefficient, open-water (10) \\
\eta_0 &= T_0 V_0 / (2 \pi n Q_0) = J_0 k_{t0} / (2 \pi k_{q0}): propeller efficiency, open-water (11)
\end{aligned}
$$

where $n$ is the propeller rotational velocity and $D$ the propeller diameter.

We also use the following notation for the dimensionless variables for the propeller operation behind ship (self-propulsion conditions):

$$
\begin{aligned}
J &= V / (n D) : advance coefficient, self-propulsion (12) \\
k_t &= T / (\rho n^2 D^4) : thrust coefficient, self-propulsion (13) \\
k_q &= Q / (\rho n^2 D^5) = DHP / (2 \pi p n^3 D^5): torque coefficient, self-propulsion (14) \\
\eta &= T V / (2 \pi n Q) = J k_{t0} / (2 \pi k_q): propeller efficiency, self-propulsion (15)
\end{aligned}
$$

where $DHP = 2 \pi n Q$ the delivered horsepower of the propeller operating behind ship.

Due to equations (5) to (7), dimensionless variables for propeller operation separate from ship and behind ship, are related as follows:

$$
\begin{aligned}
J_0 &= J (1 - w) <-> J = J_0 / (1 - w) \\
k_{t0} &= k_t \\
k_{q0} &= \eta_R k_q <-> k_q = k_{q0} / \eta_R \\
\eta_0 &= \eta (1 - w) / \eta_R <-> \eta = \eta_0 \eta_R / (1 - w)
\end{aligned}
$$

In [1]:
import numpy as np
import csv

def _estimate_bseries_poly_coeffs(
    prop_PD,    # pitch/diameter ratio P/D [-]
    prop_AEA0,  # expanded area ratio AE/A0 [-]
    prop_Z,     # blade count [-]
    Rn=2e7,     # target Reynolds number for which to embed the correction
):
    """
    Returns Wageningen B-series K_T(J) and K_Q(J) polynomial coefficients with
    Reynolds-number correction embedded for a given Rn.

    Base K_T(J) and K_Q(J) are from the 2e6 Reynolds table:
        K_T(J) = a0 + a1*J + a2*J^2 + ...
        K_Q(J) = b0 + b1*J + b2*J^2 + ...

    The additional Reynolds-number effect above 2e6 is added as ΔK_T(J) and ΔK_Q(J),
    which are polynomials in J (up to J^2) for fixed PD, AE/A0, Z and Rn.
    """

    def load_terms(filepath):
        terms = []
        with open(filepath, newline='') as f:
            reader = csv.reader(f)
            header = next(reader)  # skip header
            for row in reader:
                # Expecting: C, s, t, u, v (all floats or ints)
                C, s, t, u, v = map(float, row)
                terms.append([C, int(s), int(t), int(u), int(v)])
        return np.array(terms)

    def polynomial_in_J(terms, PD, AEA0, Z):
        # Highest possible J exponent
        max_s = int(np.max(terms[:, 1]))
        coeffs = np.zeros(max_s + 1)

        for C, s, t, u, v in terms:
            s = int(s)
            t = int(t)
            u = int(u)
            v = int(v)
            coeffs[s] += C * (PD**t) * (AEA0**u) * (Z**v)

        return coeffs

    def _delta_KT_coeffs(PD, AEA0, Z, Rn):
        """
        Returns (c0, c1, c2) such that
            ΔK_T(J) = c0 + c1*J + c2*J^2
        for given PD, AE/A0, Z and Rn, using the polynomial given in the paper.
        """
        K = np.log10(Rn) - 0.301

        c0 = (
            + 0.000353485
        )
        c1 = (
            - 0.00478125 * AEA0 * PD
            + 0.0000954 * K * Z * AEA0 * PD
            + 0.0000032049 * K * Z**2 * AEA0 * PD**3
        )
        c2 = (
            - 0.00333758 * AEA0
            + 0.000257792 * K**2 * AEA0
            + 0.0000643192 * K * PD**6
            - 0.0000110636 * K**2 * PD**6
            - 0.0000276305 * K**2 * Z * AEA0
        )

        return np.array([c0, c1, c2])

    def _delta_KQ_coeffs(PD, AEA0, Z, Rn):
        """
        Returns (c0, c1, c2) such that
            ΔK_Q(J) = c0 + c1*J + c2*J^2
        for given PD, AE/A0, Z and Rn, using the polynomial given in the paper.
        """
        K = np.log10(Rn) - 0.301

        c0 = (
            - 0.000591412
            + 0.00696898 * PD
            - 0.0000666654 * Z * PD**6
            + 0.0160818 * AEA0**2
            - 0.000938091 * K * PD
            - 0.00059593 * K * PD**2
            + 0.0000782099 * K**2 * PD**2
            + 0.0000230171 * K * Z * PD**6
            - 0.00000184341 * K**2 * Z * PD**6
            - 0.00400252 * K * AEA0**2
            + 0.000220915 * K**2 * AEA0**2
        )
        c1 = - 0.00000088528 * K**2 * Z * AEA0 * PD
        c2 = + 0.0000052199 * K * Z * AEA0

        return np.array([c0, c1, c2])

    def _add_delta(base_coeffs, delta_coeffs):
        """
        base_coeffs: array_like, base polynomial in J (K_T0 or K_Q0).
        delta_coeffs: array_like, (c0,c1,c2) or (d0,d1,d2) for ΔK(J).
        Returns base_coeffs + delta_coeffs (padded as needed).
        """
        n = max(len(base_coeffs), len(delta_coeffs))
        out = np.zeros(n)
        out[:len(base_coeffs)] = base_coeffs
        out[:len(delta_coeffs)] += delta_coeffs
        return out

    # --- Base 2e6 Reynolds polynomials in J ---
    KT_TERMS = load_terms("data/kt_terms_wageningen_b_series.csv")
    KQ_TERMS = load_terms("data/kq_terms_wageningen_b_series.csv")

    base_KT = polynomial_in_J(KT_TERMS, prop_PD, prop_AEA0, prop_Z)
    base_KQ = polynomial_in_J(KQ_TERMS, prop_PD, prop_AEA0, prop_Z)

    # --- Reynolds corrections, folded into J-polynomials for this Rn ---
    dKT = _delta_KT_coeffs(prop_PD, prop_AEA0, prop_Z, Rn)
    dKQ = _delta_KQ_coeffs(prop_PD, prop_AEA0, prop_Z, Rn)

    prop_k_T_coeffs = _add_delta(base_KT, dKT)
    prop_k_Q_coeffs = _add_delta(base_KQ, dKQ)

    return prop_k_T_coeffs, prop_k_Q_coeffs

def _estimate_prop_chord_07(
    prop_D,       # propeller diameter [m]
    prop_hub_D,   # propeller hub diameter [m]
    prop_AEA0,    # expanded area ratio AE/A0 [-]
    prop_Z,       # blade count [-]
):
    prop_R = prop_D / 2.0                    # propeller radius [m]
    prop_hub_R = prop_hub_D / 2.0            # propeller hub radius [m]

    prop_A0 = np.pi * prop_R**2              # propeller disc area [m^2]
    prop_AE = prop_AEA0 * prop_A0            # total expanded blade area [m^2]
    prop_A_blade = prop_AE / prop_Z          # expanded area per blade [m^2]

    prop_L_radial = prop_R - prop_hub_R  # radial blade span [m]

    prop_c_mean = prop_A_blade / prop_L_radial      # mean chord [m]
    prop_c_07 = prop_c_mean                         # chord at 0.7R [m]

    return {
        "prop_R": prop_R,                   # [m]
        "prop_hub_R": prop_hub_R,           # [m]
        "prop_A0": prop_A0,                 # [m^2]
        "prop_AE": prop_AE,                 # [m^2]
        "prop_A_blade": prop_A_blade,       # [m^2]
        "prop_L_radial": prop_L_radial,     # [m]
        "prop_c_mean": prop_c_mean,         # [m]
        "prop_c_07": prop_c_07,             # [m]
    }


def _estimate_prop_I_r(
    prop_blade_thickness,  # blade thickness [m]
    prop_mat_rho,          # material density approximation [kg/m^3]
    prop_AE,               # expanded area total [m^2]
    prop_Z,                # blade count [-]
    prop_hub_R,            # hub radius [m]
    prop_L_radial,         # radial blade span [m]
    prop_total_m           # propeller total mass [kg]
):
    prop_blades_total_m = prop_mat_rho * prop_AE * prop_blade_thickness     # blade mass estimate [kg]
    prop_blade_m = prop_blades_total_m / prop_Z                          # mass per blade [kg]
    prop_hub_m = prop_total_m - prop_blades_total_m                      # hub mass [kg]
    if prop_hub_m < 0:
        return None

    J_hub = 0.5 * prop_hub_m * prop_hub_R**2                             # hub inertia [kg·m^2]
    J_blade = (1.0/3.0) * prop_blade_m * prop_L_radial**2                # blade inertia about root [kg·m^2]
    J_total = J_hub + prop_Z * J_blade                                   # total inertia [kg·m^2]

    return {
        "prop_blade_thickness": prop_blade_thickness,   # [m]
        "prop_mat_rho": prop_mat_rho,                         # [kg/m^3]
        "prop_blades_total_m": prop_blades_total_m,     # [kg]
        "prop_blade_m": prop_blade_m,                   # [kg]
        "prop_hub_m": prop_hub_m,                       # [kg]
        "prop_L_radial": prop_L_radial,                 # [m]
        "J_hub": J_hub,                                 # [kg·m^2]
        "J_blade": J_blade,                             # [kg·m^2]
        "J_total": J_total                              # [kg·m^2]
    }

# From https://doi.org/10.1017/9781316494196.010:
# [1] A. F. Molland, S. R. Turnock, and D. A. Hudson, “Wake and Thrust Deduction,” in Ship Resistance and Propulsion: Practical Estimation of Ship Propulsive Power, Cambridge: Cambridge University Press, 2017, pp. 149–173
def _estimate_wake_fraction(
    hull_u,      # hull speed [m/s]
    hull_L,      # hull length at waterline [m]
    hull_B,      # hull beam at waterline [m]
    hull_M,      # hull+payload mass [kg]
    hull_C_TB,     # block coefficient C_B [-]
    water_rho,   # water density [kg/m^3]
    g=9.81,      # gravity [m/s^2]
):
    hull_D = hull_M * g  # displacement weight [N]
    # Draft from C_B: ∇ = C_B * L * B * T, ∇ = hull_D / (ρ g)
    hull_disp_vol = hull_D / (water_rho * g)  # displacement volume [m^3]
    hull_T = hull_disp_vol / (hull_C_TB * hull_L * hull_B)  # draft [m]

    hull_Fn = hull_u / np.sqrt(g * hull_L)  # Froude number [-]

    # Taylor relation (Molland et al.): w_T = 0.5*C_B - 0.05  (single-screw) [-]
    hull_W = 0.5 * hull_C_TB - 0.05  # wake fraction w_T [-]

    return {
        "hull_W": hull_W,  # wake fraction w_T [-]
        "hull_Fn": hull_Fn,                        # Froude number [-]
        "hull_T": hull_T,                          # draft [m]
    }

# From https://doi.org/10.1017/9781316494196.010:
# [1] A. F. Molland, S. R. Turnock, and D. A. Hudson, “Wake and Thrust Deduction,” in Ship Resistance and Propulsion: Practical Estimation of Ship Propulsive Power, Cambridge: Cambridge University Press, 2017, pp. 149–173
def _estimate_thrust_deduction(
    hull_W,  # wake fraction w_T [-]
    k_R=0.6,             # interaction factor k_R [-]
):
    hull_T_ded = k_R * hull_W  # thrust deduction t [-]
    hull_eta_H = (1.0 - hull_T_ded) / (1.0 - hull_W)  # hull efficiency η_H [-]

    return {
        "hull_T_ded": hull_T_ded,    # thrust deduction t [-]
        "hull_k_R": k_R,           # interaction factor k_R [-]
        "hull_eta_H": hull_eta_H,  # hull efficiency η_H [-]
    }

def _round_re_to_2e_power(Re):
    """
    Round Re to the closest value in the sequence 2*10^x (x integer).
    """
    x = int(round(np.log10(Re / 2.0)))
    return 2.0 * 10.0**x

In [2]:
def estimate_initial_values(
    motor_rpm,             # motor speed [rpm]
    batt_v,                # Battery voltage [V]
    esc_i_in,                # ESC Input current [A]
    motor_eta,             # motor efficiency (shaft / electrical-in) [-]
    esc_eta,               # ESC efficiency (electrical-out / electrical-in) [-]
    trans_eta,             # transmission efficiency (prop-shaft / motor-shaft) [-]
    trans_k,               # transmission gear ratio motor:prop [-]
    hull_u,                # hull forward speed [m/s]
    hull_L,                # hull length at waterline [m]
    hull_B,                # hull beam at waterline [m]
    hull_M,                # hull+payload mass [kg]
    hull_C_TB,               # block coefficient C_B [-]
    prop_D,                # propeller diameter [m]
    prop_Z,                # blade count [-]
    prop_hub_D,            # hub diameter [m]
    prop_AEA0,             # expanded area ratio AE/A0 [-]
    prop_PD,               # pitch/diameter ratio P/D [-]
    prop_total_m,          # propeller total mass [kg]
    prop_blade_thickness,  # blade thickness estimate [m]
    prop_mat_rho,          # blade material density [kg/m^3]
    prop_c_07=None,        # (optional) chord at 70% radius [m]
    water_rho=1000.0,      # water density [kg/m^3]
    water_mu=1e-3,         # water dynamic viscosity [Pa·s]
):
    # --- Motor / prop kinematics ---

    motor_n = motor_rpm / 60.0  # motor rotational speed [1/s]
    motor_w = 2.0 * np.pi * motor_n  # motor angular speed [rad/s]

    prop_rpm = motor_rpm * trans_k  # propeller speed [rpm]
    prop_n = prop_rpm / 60.0  # propeller rotational speed [1/s]
    prop_w = 2.0 * np.pi * prop_n  # propeller angular speed [rad/s]

    # --- Hull wake and advance speed (Taylor w_T from C_B) ---

    wake = _estimate_wake_fraction(
        hull_u=hull_u,
        hull_L=hull_L,
        hull_B=hull_B,
        hull_M=hull_M,
        hull_C_TB=hull_C_TB,
        water_rho=water_rho,
    )
    hull_W = wake["hull_W"]  # wake fraction w_T [-]
    hull_Fn = wake["hull_Fn"]  # Froude number [-]
    hull_T = wake["hull_T"]  # draft [m]

    prop_va = hull_u * (1.0 - hull_W)  # prop advance speed [m/s]

    prop_I_ra = prop_va / (prop_n * prop_D)  # advance coefficient Ja [-]

    # --- Thrust deduction and hull efficiency ---

    td = _estimate_thrust_deduction(
        hull_W=hull_W,
        k_R=0.6,  # initial guess for small craft [-]
    )
    hull_T_ded = td["hull_T_ded"]    # thrust deduction t [-]
    hull_k_R = td["hull_k_R"]      # interaction factor k_R [-]
    hull_eta_H = td["hull_eta_H"]  # hull efficiency η_H [-]

    # --- Geometry and Reynolds at 0.7R ---

    geom = _estimate_prop_chord_07(
        prop_D=prop_D,
        prop_hub_D=prop_hub_D,
        prop_AEA0=prop_AEA0,
        prop_Z=prop_Z,
    )

    prop_R = geom["prop_R"]  # propeller radius [m]
    prop_hub_R = geom["prop_hub_R"]  # propeller hub radius [m]
    prop_A0 = geom["prop_A0"]  # propeller disc area [m^2]
    prop_AE = geom["prop_AE"]  # total expanded blade area [m^2]
    prop_A_blade = geom["prop_A_blade"]  # expanded area per blade [m^2]
    prop_L_radial = geom["prop_L_radial"]  # radial blade span [m]
    prop_c_mean = geom["prop_c_mean"]  # mean chord [m]
    prop_c_07_est = geom["prop_c_07"]  # chord at 0.7R [m]

    # Use measured chord if provided, otherwise the estimated one
    if prop_c_07 is None:
        prop_c_07 = prop_c_07_est

    prop_r_07 = 0.7 * prop_R  # radius at 0.7R [m]
    prop_Vtheta_07 = prop_w * prop_r_07  # tangential velocity at 0.7R [m/s]
    prop_Vrel_07 = np.sqrt(prop_va**2 + prop_Vtheta_07**2)  # relative velocity at 0.7R [m/s]

    prop_Re_07 = water_rho * prop_Vrel_07 * prop_c_07 / water_mu  # Reynolds at 0.7R [-]
    prop_Re_07_rounded = _round_re_to_2e_power(prop_Re_07)

    # --- k_T, k_Q polynomials ---

    prop_k_T_coeffs, prop_k_Q_coeffs = _estimate_bseries_poly_coeffs(
        prop_PD=prop_PD,
        prop_AEA0=prop_AEA0,
        prop_Z=prop_Z,
        # Rn=prop_Re_07_rounded,
        Rn=prop_Re_07,
    )

    # TODO: Relative rotative efficiency
    # prop_eta_R = _estimate_propeller_rotative_efficiency(prop_AE/prop_A0, )

    def _eval_poly(coeffs, x):
        result = 0.0  # Horner accumulator [-]
        for c in reversed(coeffs[1:]):
            result = (result + c) * x  # Horner step [-]
        return result + coeffs[0]

    prop_k_T = _eval_poly(prop_k_T_coeffs, prop_I_ra)  # thrust coefficient [-]
    prop_k_Q = _eval_poly(prop_k_Q_coeffs, prop_I_ra)  # torque coefficient [-]

    # --- Hydrodynamic loads and shaft power ---

    prop_T = water_rho * (prop_n**2) * (prop_D**4) * prop_k_T  # thrust T [N]
    prop_Q = water_rho * (prop_n**2) * (prop_D**5) * prop_k_Q  # torque Q [N·m]

    prop_P_shaft = 2.0 * np.pi * prop_n * prop_Q  # prop shaft power from hydro [W]

    prop_eta_open = None  # open-water efficiency [-]
    if prop_k_Q != 0.0:
        prop_eta_open = prop_I_ra * prop_k_T / (2.0 * np.pi * prop_k_Q)  # open-water efficiency [-]

    # --- Electrical and mechanical power chain ---

    esc_P_in = batt_v * esc_i_in
    esc_P_out = esc_P_in * esc_eta  # electrical power into motor [W]
    motor_P_shaft = esc_P_out * motor_eta  # motor shaft power [W]
    trans_P_out = motor_P_shaft * trans_eta  # power at prop shaft [W]

    system_eta_elec_to_prop = (
        prop_P_shaft / esc_P_in if esc_P_in > 0 else None
    )  # battery-elec -> prop hydro [-]

    system_eta_available_to_prop = (
        prop_P_shaft / trans_P_out if trans_P_out > 0 else None
    )  # hydro / available shaft [-]

    # --- Propeller inertia estimation ---

    J_est = _estimate_prop_I_r(
        prop_blade_thickness=prop_blade_thickness,
        prop_mat_rho=prop_mat_rho,
        prop_AE=prop_AE,
        prop_Z=prop_Z,
        prop_hub_R=prop_hub_R,
        prop_L_radial=prop_L_radial,
        prop_total_m=prop_total_m,
    )

    return {
        # Motor / ESC / transmission
        "motor_rpm": motor_rpm,  # motor speed [rpm]
        "motor_n": motor_n,  # motor rotational speed [1/s]
        "motor_w": motor_w,  # motor angular speed [rad/s]
        "motor_v": batt_v,  # motor DC voltage [V]
        "motor_i": esc_i_in,  # motor DC current [A]
        "motor_eta": motor_eta,  # motor efficiency [-]
        "esc_eta": esc_eta,  # ESC efficiency [-]
        "trans_eta": trans_eta,  # transmission efficiency [-]
        "trans_k": trans_k,  # transmission gear ratio motor:prop [-]
        "esc_P_in": esc_P_in,  # electrical input power [W]
        "esc_P_out": esc_P_out,  # ESC output power to motor [W]
        "motor_P_shaft": motor_P_shaft,  # motor shaft power [W]
        "trans_P_out": trans_P_out,  # available prop shaft power [W]
        "system_eta_elec_to_prop": system_eta_elec_to_prop,  # overall eff battery->prop hydro [-]
        "system_eta_available_to_prop": system_eta_available_to_prop,  # prop hydro / available shaft [-]

        # Hull
        "hull_u": hull_u,  # hull speed [m/s]
        "hull_L": hull_L,  # hull length [m]
        "hull_B": hull_B,  # hull beam [m]
        "hull_M": hull_M,  # hull+payload mass [kg]
        "hull_C_TB": hull_C_TB,  # [-]
        "hull_T": hull_T,  # estimated draft [m]
        "hull_Fn": hull_Fn,  # Froude number [-]
        "hull_W": hull_W,  # hull wake fraction [-]
        "hull_T_ded": hull_T_ded,  # hull thrust deduction [-]
        "hull_k_R": hull_k_R,  # [-]
        "hull_eta_H": hull_eta_H,  # [-]

        # Propeller kinematics
        "prop_rpm": prop_rpm,  # propeller speed [rpm]
        "prop_n": prop_n,  # propeller rotational speed [1/s]
        "prop_w": prop_w,  # propeller angular speed [rad/s]
        "prop_va": prop_va,  # advance speed at prop [m/s]
        "prop_I_ra": prop_I_ra,  # advance coefficient Ja [-]

        # Propeller geometry
        "prop_D": prop_D,  # propeller diameter [m]
        "prop_R": prop_R,  # propeller radius [m]
        "prop_Z": prop_Z,  # blade count [-]
        "prop_hub_D": prop_hub_D,  # hub diameter [m]
        "prop_hub_R": prop_hub_R,  # hub radius [m]
        "prop_A0": prop_A0,  # propeller disc area [m^2]
        "prop_AE": prop_AE,  # total expanded blade area [m^2]
        "prop_A_blade": prop_A_blade,  # expanded area per blade [m^2]
        "prop_L_radial": prop_L_radial,  # radial blade span [m]
        "prop_c_mean": prop_c_mean,  # mean chord [m]
        "prop_c_07": prop_c_07,  # chord at 0.7R [m]
        "prop_r_07": prop_r_07,  # radius at 0.7R [m]
        "prop_Vtheta_07": prop_Vtheta_07,  # tangential velocity at 0.7R [m/s]
        "prop_Vrel_07": prop_Vrel_07,  # relative velocity at 0.7R [m/s]
        "prop_Re_07": prop_Re_07,  # Reynolds at 0.7R [-]

        # k_T, k_Q and loads
        "prop_k_T_coeffs": prop_k_T_coeffs,  # k_T polynomial coefficients [-]
        "prop_k_Q_coeffs": prop_k_Q_coeffs,  # k_Q polynomial coefficients [-]
        "prop_k_T": prop_k_T,  # thrust coefficient [-]
        "prop_k_Q": prop_k_Q,  # torque coefficient [-]
        "prop_T": prop_T,  # thrust T [N]
        "prop_Q": prop_Q,  # torque Q [N·m]
        "prop_P_shaft": prop_P_shaft,  # prop shaft power from hydro [W]
        "prop_eta_open": prop_eta_open,  # open-water efficiency [-]

        # Inertia
        "prop_I_r": J_est["J_total"] if J_est is not None else None,  # inertia J [kg·m^2]
        "prop_I_r_details": J_est,  # inertia breakdown dict [kg·m^2]

        # Fluid properties
        "water_rho": water_rho,  # water density [kg/m^3]
        "water_mu": water_mu,  # water dynamic viscosity [Pa·s]
    }

In [3]:
# Getting system operating point:
# df_train = df.loc[(df.index >= train_section['start']) & (df.index <= train_section['end'])][130:] # This reduced section also works well
# v = df_train['batt_v'].to_numpy()
# i = df_train['esc_i_in'].to_numpy()
# w = df_train['motor_w'].to_numpy()
# k = (v*i).argmax()
# x = df_train.iloc[k-22:k+21]
# print(x.mean())
# x.plot(subplots=True)
# plt.show()

# Result:
# batt_v         32.329286
# pilot_d         1.000000
# motor_w       282.781391
# esc_i_in        126.432603
# hull_u          4.879954

In [4]:
# ESC Efficiency at 100% Duty-cyle
esc_mosfet_rds_on = 2.3e-3*2.0 # IXFN420N10T @ 150C
esc_i_in=126.432603
batt_v = 32.329286
esc_P_loss = esc_mosfet_rds_on * esc_i_in**2
esc_P_in = esc_i_in * batt_v
esc_P_out = esc_P_in - esc_P_loss
esc_eta = esc_P_out / esc_P_in
esc_eta

0.9820104293735407

In [5]:
# Motor Efficiency
esc_D = 1
motor_R = 0.01953449524955559
motor_B = 0.0012072185526191795
motor_w = 282.781391
motor_i = esc_i_in * esc_D
motor_P_in = batt_v * esc_i_in
motor_P_loss = motor_B * motor_w**2  + motor_R * motor_i**2
motor_P_out = motor_P_in - motor_P_loss
motor_eta = motor_P_out / motor_P_in
motor_eta

0.8999875454221669

In [ ]:
in_to_m = 0.0254
radps_to_rpm = 60 / (2 * np.pi)

vals = estimate_initial_values(
    motor_rpm=282.781391*radps_to_rpm,   # motor speed [rpm]
    batt_v=32.329286,                    # motor voltage [V]
    esc_i_in=126.432603,                   # motor current [A]
    motor_eta=0.8999875454221669,        # motor efficiency at motor_v and motor_i

    esc_eta=0.9820104293735407,          # ESC efficiency

    trans_eta=0.95,                      # transmission efficiency
    trans_k=14/22,                       # input:output transmission gear ratio [-]

    hull_u=4.879954,                     # hull speed [m/s]
    hull_L=6.0,                          # hull length [m]
    hull_B=0.959,                        # hull beam, waterline [m]
    hull_M=292.0,                        # hull+payload mass [kg]
    hull_C_TB=0.385,                     # block coefficient [-]

    prop_D=9*in_to_m,                    # measured prop diameter 215 mm [m]
    prop_Z=3,                            # blade count [-]
    prop_hub_D=0.06,                     # measured hub diameter 60 mm [m]
    prop_AEA0=0.242,                     # AE/A0 estimate from the image [-]
    prop_PD=10.5/9,                      # P/D from 10.5" pitch and 9" D [-]
    prop_total_m=0.66,                   # measured prop mass [kg]
    prop_blade_thickness=0.005,          # blade thickness guess 5 mm [m]
    prop_mat_rho=2700.0,                 # aluminium density [kg/m^3]
    prop_c_07=9.29/100,                  # propeller chord measured at 70% radius [m]

    water_rho=1023.0,                    # fresh water density [kg/m^3]
    water_mu=0.0009268,                  # dynamic viscosity [Pa·s]
)

print("'prop_k_T_coeffs': np.array(["+ ', '.join([str(a) for a in vals['prop_k_T_coeffs']]) + '])')
print("'prop_k_Q_coeffs': np.array(["+ ', '.join([str(a) for a in vals['prop_k_Q_coeffs']]) + '])')

display(vals['hull_T_ded'])
display(vals['hull_W'])

'prop_k_T_coeffs': np.array([0.3720820781440099, -0.10825004156370191, -0.1673547552114919, 0.02821403437779094])
'prop_k_Q_coeffs': np.array([0.05470368178162633, -0.017535689093141415, -0.0005099909931731473, -0.010783064763591884])


0.0855

0.14250000000000002

In [11]:
vals

{'motor_rpm': 2700.3633715231203,
 'motor_n': 45.00605619205201,
 'motor_w': 282.781391,
 'motor_v': 32.329286,
 'motor_i': 126.432603,
 'motor_eta': 0.8999875454221669,
 'esc_eta': 0.9820104293735407,
 'trans_eta': 0.95,
 'trans_k': 0.6363636363636364,
 'esc_P_in': 4087.4757821114586,
 'esc_P_out': 4013.9438478452225,
 'motor_P_shaft': 3612.4994710846295,
 'trans_P_out': 3431.874497530398,
 'system_eta_elec_to_prop': 0.9333985454389458,
 'system_eta_available_to_prop': 1.1117084707745666,
 'hull_u': 4.879954,
 'hull_L': 6.0,
 'hull_B': 0.959,
 'hull_M': 292.0,
 'hull_C_TB': 0.385,
 'hull_T': 0.1288476881638135,
 'hull_Fn': 0.6360710056235633,
 'hull_W': 0.14250000000000002,
 'hull_T_ded': 0.0855,
 'hull_k_R': 0.6,
 'hull_eta_H': 1.0664723032069972,
 'prop_rpm': 1718.413054605622,
 'prop_n': 28.640217576760367,
 'prop_w': 179.95179427272728,
 'prop_va': 4.184560554999999,
 'prop_I_ra': 0.6391419420445708,
 'prop_D': 0.2286,
 'prop_R': 0.1143,
 'prop_Z': 3,
 'prop_hub_D': 0.06,
 'prop_h